In [47]:
import cv2
import numpy as np
import dlib
import matplotlib.pyplot as plt
import pandas as pd

In [48]:
for i in range(100):

    face_path = './dataset/train/img/%s.png'%i
    face_origin = cv2.imread(face_path)
    face_copy = face_origin.copy()
    face_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    detector = dlib.get_frontal_face_detector()
    dlib_rects = detector(face_rgb,1)
    
    img_show_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)

    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    globals()['img'+str(i)] = []
    
    for dlib_rect in dlib_rects:
        points = predictor(face_rgb, dlib_rect)
        list_points = list(map(lambda p: [p.x, p.y], points.parts()))
        globals()['img'+str(i)].append(list_points)
        globals()['img'+str(i)] = np.array(globals()['img'+str(i)])

In [8]:
# xyxyxyxyxyxyxy(0~16)
for i in range(100):
    globals()['img'+str(i)] = globals()['img'+str(i)][0][:17]
    globals()['img'+str(i)] = globals()['img'+str(i)].flatten()

In [9]:
train_x = np.zeros((100, 34))
for i in range(100):
    for j in range(34):
        train_x[i][j] = globals()['img'+str(i)][j]

In [10]:
for i in range(100):

    face_path = './dataset/test/img/%s.png'%i
    face_origin = cv2.imread(face_path)
    face_copy = face_origin.copy()
    face_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    detector = dlib.get_frontal_face_detector()
    dlib_rects = detector(face_rgb,1)
    
    img_show_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)

    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    globals()['img_'+str(i)] = []
    
    for dlib_rect in dlib_rects:
        points = predictor(face_rgb, dlib_rect)
        list_points = list(map(lambda p: [p.x, p.y], points.parts()))
        globals()['img_'+str(i)].append(list_points)
        globals()['img_'+str(i)] = np.array(globals()['img_'+str(i)])

In [11]:
# xyxyxyxyxyxyxy(0~16)
for i in range(100):
    globals()['img_'+str(i)] = globals()['img_'+str(i)][0][:17]
    globals()['img_'+str(i)] = globals()['img_'+str(i)].flatten()
    
test_x = np.zeros((100, 34))
for i in range(100):
    for j in range(34):
        test_x[i][j] = globals()['img_'+str(i)][j]

In [38]:
train_y = pd.read_csv('./dataset/train/train.csv')
train_y = train_y.iloc[:, 5:7]
test_y = pd.read_csv('./dataset/test/test.csv')
test_y = test_y.iloc[:, 5:7]

In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

model = LinearRegression()
model.fit(train_x, train_y)
y_pred = model.predict(train_x)
mae = mean_absolute_error(train_y, y_pred)

mae

0.0843492860361374

In [40]:
y_pred = model.predict(test_x)
mae = mean_absolute_error(test_y, y_pred)

mae

0.09983615513773433

In [41]:
model.intercept_, model.coef_

(array([ -0.50743233, -34.14507061]),
 array([[-0.00437459,  0.01177658, -0.01192196, -0.03158696,  0.02718324,
          0.02204035, -0.04681052,  0.03725349, -0.02492314,  0.01529143,
          0.08302979, -0.11272703, -0.0167061 ,  0.01659942, -0.02618197,
          0.03345795,  0.03141358, -0.05145365,  0.01284412,  0.04018401,
          0.02915692, -0.03256804, -0.05303529,  0.06694477,  0.00650238,
         -0.03010463,  0.04136678, -0.02346561, -0.03619642,  0.01640819,
         -0.02224511,  0.01858691,  0.03934565, -0.01610522],
        [ 0.01116581, -0.01362386,  0.00243284, -0.02998978,  0.03954541,
         -0.00669718, -0.00361351,  0.06312499, -0.01921339,  0.05204097,
         -0.03916485, -0.05125232,  0.0028919 , -0.02140422,  0.03557921,
         -0.01147331, -0.01043128, -0.03446234,  0.0165871 ,  0.06122424,
          0.03645928, -0.009709  , -0.06543319,  0.01414299,  0.02655865,
          0.00961949,  0.02522219, -0.00897345, -0.01804209,  0.00973467,
          0.

In [44]:
prediction = pd.DataFrame(y_pred)
prediction.columns = ['jawsPosition','jawsSize']
prediction

,jawsPosition,jawsSize
0,0.291378,0.610664
1,0.351801,0.719208
2,0.447414,0.572736
3,0.406243,0.572073
4,0.319585,0.362907
...,...,...
95,0.515153,0.801123
96,0.426061,0.411668
97,0.646365,0.424560
98,0.578473,0.324780


In [43]:
test_y

,jawsPosition,jawsSize
0,0.323832,0.595424
1,0.252126,0.766253
2,0.466237,0.531876
3,0.289253,0.519737
4,0.225629,0.502349
...,...,...
95,0.320538,0.766127
96,0.530781,0.438227
97,0.565084,0.416299
98,0.569411,0.301160


In [46]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(test_y, prediction)**0.5
print(rmse)

0.12469882397879001
